In [17]:
import re
import time

FILENAME="input.txt"
#FILENAME="test.txt"

# V2 - baby step, giant step

https://docs.python.org/3/howto/regex.html
https://docs.python.org/3/library/re.html#checking-for-a-pair

In [38]:
str_test = "12 34 56"
searched = re.search("^(?P<tgt_first>\d+) (?P<src_first>\d+) (?P<item_count>\d+)$",str_test)
tgt_first = int(searched.group(1))
src_first = int(searched.group(2))
item_count = int(searched.group(3))
print(tgt_first,src_first,item_count)
print(searched.groupdict())
print(searched.group("src_first"))

12 34 56
{'tgt_first': '12', 'src_first': '34', 'item_count': '56'}
34


In [56]:
str_test = "12 24"
searched = re.search(r"(\d+) \1",str_test)
print(searched)

str_test = "12 12"
searched = re.search(r"(\d+) \1",str_test)
print(searched)

<re.Match object; span=(1, 4), match='2 2'>
<re.Match object; span=(0, 5), match='12 12'>


In [18]:
def parseFile(fileName):

    state=0 #-1 seeds 0-empty line 1-header 2-numbers

    with open(fileName) as file:
        #1. get seeds
        searched = re.search("^seeds:(.*)$",file.readline())
        seed_list = [int(x) for x in searched.group(1).split()]
    
        #print(result_dict)
        state=-1
    
        transforms_all = []
        transforms_curr = []
    
        for ll in file:
            if ll.strip()=="":
                state = 0
            else:
                if state not in [0,1,2]:
                    pass
                elif state==0:
                    #starting next group
                    searched = re.search("^(\w+)-to-(\w+) map:$",ll)
                    state=1
                    if len(transforms_curr) > 0:
                        transforms_all.append(transforms_curr)
                    transforms_curr = []
                else:
                    searched = re.search("^(\d+) (\d+) (\d+)$",ll)
                    tgt_first = int(searched.group(1))
                    src_first = int(searched.group(2))
                    item_count = int(searched.group(3))
                
                    transforms_curr.append((tgt_first,src_first,item_count))
                    state=2

        transforms_all.append(transforms_curr)
    return (seed_list,transforms_all)
                


In [19]:
def processSingleSeed(seedID,transforms):
    seed_mappedTo = seedID
    for transf_sgl in transforms:        
        for tgt_first,src_first,item_count in transf_sgl:      
            if seed_mappedTo >= src_first and (seed_mappedTo < src_first + item_count):
                seed_mappedTo = tgt_first + (seed_mappedTo-src_first)
                break
    return seed_mappedTo

In [20]:
def getSeedsWithCount(seeds_mixed):
    seed_count = {}
    tmp_s = -1
    tmp_count=-1
    
    for x in seeds_mixed:
        x_int = int(x)
        if tmp_count != 0:
            tmp_s = x_int
            tmp_count = 0
        else:
            seed_count[tmp_s]=x_int
            tmp_count = 1
    
    return seed_count
#getSeedsWithCount(seeds2)

In [21]:
def getAllSeeds(seeds_centre,negative_count,positive_count,negative_limit,positive_limit,step=1):
    """
    [seeds_centre-negative_count,seeds_centre+positive_count] subject to limits
    """
    res = []
    positive_count = max(positive_count,0)
    x_neg = seeds_centre - negative_count
    x_neg = max(x_neg,negative_limit)
    res = [y for y in range(x_neg,seeds_centre,step)]
    x_pos = seeds_centre+positive_count
    x_pos = min(x_pos,positive_limit)
    res += [y for y in range(seeds_centre,x_pos+1,step)]
    
    return res
print(getAllSeeds(10,9,9,5,11))
print(getAllSeeds(11,0,10,0,23))

[5, 6, 7, 8, 9, 10, 11]
[11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21]


In [22]:
MY_STEP = 100000

In [23]:
def getSeedsCoarse(seeds_count,add_last=False):
    res = []
    for y,l in seeds_count.items():
        res += getAllSeeds(y,0,l-1,0,y+l-2,MY_STEP)
        if add_last:
            res+=[y+l-1]
    return res
#res = getSeedsCoarse(seeds2_count)
#print(len(res))

In [24]:
seeds2_mixed, transf_all = parseFile(FILENAME)
seeds2_count = getSeedsWithCount(seeds2_mixed)
seeds2_coarse = getSeedsCoarse(seeds2_count,True)
print("seeds2_count lenght:",len(seeds2_count))
print()
print("transforms length:",len(transf_all))
print()
print("seeds2_coarse length: ",len(seeds2_coarse))
#print(seeds2_coarse)

seeds2_count lenght: 10

transforms length: 7

seeds2_coarse length:  24474


In [25]:
#processSingleSeed(13,transf_all)

In [26]:
def getMinResultsFromSeedList(seeds_in_list, transf_in):

    seed_map = {}
    seed_min_idx = -1
    seed_min_val = -1

    start_time = time.time()
    for x in seeds_in_list:
        res = processSingleSeed(x,transf_in)

        #if counter//10000000 == counter/10000000:
        #    print("...")
        #elif counter//1000000 == counter/1000000:
        #    print("..")
        #elif counter//100000 == counter/100000:
        #    print(".")
        if seed_min_idx < 0 or res < seed_min_val:
            seed_min_idx = x
            seed_min_val = res

    print(">> time >>",time.time()-start_time)
    print("Step {}: {} ---> {}".format(MY_STEP,seed_min_idx,seed_min_val))
    return(seed_min_idx,seed_min_val)

seed_min_idx_coarse,seed_min_val_coarse =  getMinResultsFromSeedList(seeds2_coarse,transf_all)

>> time >> 0.6726129055023193
Step 100000: 3439892843 ---> 46358428


In [27]:
x_found = -1000000
l_found = -1000000
for x,l in seeds2_count.items():
    if seed_min_idx_coarse >= x and seed_min_idx_coarse < x+l:
        print("start: {}, length: {}, max in range: {}".format(x,l,x+l-1))
        x_found = x
        l_found = l
        break

rng_limit_lower = x_found
rng_limit_upper = x_found+l_found-1

start: 3424292843, length: 82110297, max in range: 3506403139


In [28]:

seeds2_granular = getAllSeeds(seed_min_idx_coarse,MY_STEP,MY_STEP,rng_limit_lower,rng_limit_upper)
print(min(seeds2_granular),max(seeds2_granular))

3439792843 3439992843


In [29]:
print("Granular seeds length:",len(seeds2_granular))

Granular seeds length: 200001


In [30]:
getMinResultsFromSeedList(seeds2_granular,transf_all)
# output in format (inital seed number, location number)

>> time >> 3.207167387008667
Step 100000: 3439828590 ---> 46294175


(3439828590, 46294175)